In [1]:
import numpy as np
from urllib import request
import gzip
import pickle
from tqdm import tqdm

def load():
    with open("data.pkl",'rb') as f:
        mnist = pickle.load(f)
    return mnist["training_images"], mnist["training_labels"], mnist["test_images"], mnist["test_labels"]

In [2]:
train_images, train_labels, test_images, test_labels = load()

FileNotFoundError: [Errno 2] No such file or directory: 'data.pkl'

In [3]:
train_images = train_images.reshape(-1, 28, 28)#[:5000]
test_images = test_images.reshape(-1, 28, 28)#[:2000]

In [4]:
train_labels = train_labels#[:5000]
test_labels = test_labels#[:2000]

In [5]:
from PIL import Image
Image.fromarray(train_images[0].astype(np.uint8))

In [6]:
### Ex1
def binary2decimal(binary):
    return sum(val * (2 ** idx) for idx, val in enumerate(reversed(binary)))

def f_ex1(image, d=3):
    h, w = image.shape
    size_left_right = (d - 1) // 2
    size_up_down = (d - 1) // 2
    padded_image = np.pad(image, ((size_up_down, size_up_down), (size_left_right, size_left_right)), constant_values=-1)
    results = []
    
    for i in range(size_up_down, h):
        for j in range(size_left_right, w):
            output_representation = np.zeros((d * d) - 1)
            patch = padded_image[i - size_left_right : i + size_left_right + 1, j - size_up_down : j + size_up_down + 1]
            value = padded_image[i][j]
            comparison = (patch > value) * 1
            result = comparison.reshape(-1)

            # scoatem elementul din mijloc (adica elementul pe care il comparam)
            output_representation[:len(output_representation)//2] = result[:len(output_representation)//2]
            output_representation[len(output_representation)//2:] = result[len(output_representation)//2 + 1:]
            results.append(output_representation)
 
    histogram = np.zeros((2 ** (d * d - 1)))
    for result in results:
        position = binary2decimal(result)
        histogram[int(position)] += 1
    return histogram

def process_data(images):
    results = []
    for image in tqdm(images):
        hist = f_ex1(image)
        results.append(hist)
    return np.array(results)

train_hist = process_data(train_images)

100%|██████████| 5000/5000 [00:57<00:00, 87.34it/s]


In [7]:
test_hist = process_data(test_images)

100%|██████████| 2000/2000 [00:22<00:00, 87.92it/s]


In [8]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

clf = SVC()
scaler = StandardScaler()
scaler.fit(train_hist)
train_hist_scaled = scaler.transform(train_hist)
test_hist_scaled = scaler.transform(test_hist)
clf.fit(train_hist_scaled, train_labels)

SVC()

In [9]:
clf.score(test_hist_scaled, test_labels)

0.7435

In [10]:
train_hist[0].shape

(256,)

In [11]:
### Ex2
import cv2

def compute_gradient_magnitude(image):
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    grad_mag = np.sqrt(np.square(sobelx) + np.square(sobely))
    return grad_mag

def compute_gradient_magnitude2(image):
    h, w = image.shape
    padded_image = np.pad(image, pad_width=1, mode='constant', constant_values=-1)
    grad_mag = np.zeros((h, w))

    for i in range(1, h+1):
        for j in range(1, w+1):
            gx = padded_image[i, j-1] - padded_image[i, j]  # gradient in x direction
            gy = padded_image[i-1, j] - padded_image[i, j]  # gradient in y direction
            grad_mag[i-1, j-1] = np.sqrt(gx**2 + gy**2)  # magnitude of gradient

    return grad_mag

def compute_gradient_magnitude3(image):
    h, w = image.shape
    grad_mag = np.zeros((h, w))

    for i in range(h-1):
        for j in range(w-1):
            gx = image[i+1, j] - image[i, j]  # gradient in x direction
            gy = image[i, j+1] - image[i, j]  # gradient in y direction
            grad_mag[i, j] = np.sqrt(gx**2 + gy**2)  # magnitude of gradient

    return grad_mag

def process_images(images):
    results = []
    for image in tqdm(images):
        results.append(compute_gradient_magnitude(image))
    return np.array(results)

train_grad_mag = process_images(train_images)

100%|██████████| 5000/5000 [00:00<00:00, 26000.51it/s]


In [12]:
test_grad_mag = process_images(test_images)

100%|██████████| 2000/2000 [00:00<00:00, 34553.58it/s]


In [13]:
def get_top_k_regions(grad_mag, original_image, k=30, d=4):
    patches_mag = grad_mag.reshape(-1, d, d)
    original_image = original_image.reshape(-1, d, d)
    patches_mag = np.mean(patches_mag, axis=(1, 2))
    sorted_indices = np.argsort(patches_mag)
    output = []
    for i in range(original_image.shape[0]):
        if i not in sorted_indices[-k:]:
            continue
        output.append(original_image[i])
    return np.array(output).reshape(-1)

def transform_images(images, grad_mags):
    results = []
    for i, image in enumerate(tqdm(images)):
        results.append(get_top_k_regions(grad_mags[i], image))
    return np.array(results)

ex2_train_data = transform_images(train_images, train_grad_mag)
ex2_test_data = transform_images(test_images, test_grad_mag)

100%|██████████| 2000/2000 [00:00<00:00, 3535.51it/s]


In [15]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

clf = SVC(C=10)
clf.fit(ex2_train_data, train_labels)

SVC(C=10)

In [16]:
clf.score(ex2_test_data, test_labels)

0.8565

In [18]:
# Ex3
def compute_gradient_direction(image):
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    grad_direction = np.arctan2(sobely, sobelx)
    return grad_direction

def process_images(images):
    results = []
    for image in tqdm(images):
        results.append(compute_gradient_direction(image))
    return np.array(results)

train_grad_direction = process_images(train_images)
test_grad_direction = process_images(test_images)
    
def non_max_suppression_gradient(img, direction):
    h, w = img.shape
    output = np.zeros((h, w), dtype=np.int32)
    angle = direction * 180. / np.pi
    angle[angle < 0] += 180

    for i in range(1, h-1):
        for j in range(1, w-1):
            neighbour1 = 255
            neighbour2 = 255
                
            if (0 <= angle[i,j] < 22.5) or (157.5 <= angle[i,j] <= 180):
                neighbour1 = img[i, j+1]
                neighbour2 = img[i, j-1]
  
            elif (22.5 <= angle[i,j] < 67.5):
                neighbour1 = img[i+1, j-1]
                neighbour2 = img[i-1, j+1]

            elif (67.5 <= angle[i,j] < 112.5):
                neighbour1 = img[i+1, j]
                neighbour2 = img[i-1, j]

            elif (112.5 <= angle[i,j] < 157.5):
                neighbour1 = img[i-1, j-1]
                neighbour2 = img[i+1, j+1]

            if (img[i,j] >= neighbour1) and (img[i,j] >= neighbour2):
                output[i,j] = img[i,j]
            else:
                output[i,j] = 0
    
    return output

def process_image(mags, directions):
    results = []
    for i, mag in enumerate(mags):
        results.append(non_max_suppression_gradient(mag, directions[i]))
        
    return np.array(results)

new_train_mags = process_image(train_grad_mag, train_grad_direction)
new_test_mags = process_image(test_grad_mag, test_grad_direction)

100%|███████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 32797.59it/s]


In [19]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

clf = SVC(C=10)
clf.fit(new_train_mags.reshape(-1, 784), train_labels)

SVC(C=10)

In [20]:
clf.score(new_test_mags.reshape(-1, 784), test_labels)

0.8505

In [18]:
a = np.random.randint(low=0, high=10, size=(10,10))
print(a)
print(a[-1:3, 0:2])

[[9 8 1 4 3 0 3 6 7 4]
 [8 3 2 4 4 8 7 8 1 0]
 [4 5 5 0 8 5 0 7 7 8]
 [5 7 2 1 6 7 4 5 7 9]
 [2 1 0 9 6 9 1 0 1 5]
 [5 9 8 4 5 0 9 9 4 9]
 [6 4 0 3 8 7 5 9 7 3]
 [4 6 2 5 4 6 6 0 8 3]
 [3 1 2 8 3 2 1 1 2 5]
 [5 5 9 8 4 6 2 2 9 2]]
[]


In [21]:
## Ex 4
def binarize_image(image, d=4):
    patches = image.reshape(-1, d, d)
    outputs = []
    for patch in patches:
        if d % 2 == 0:
            output_representation = np.zeros((d*d))
            value = np.mean(patch[d//2-1: d//2+1,d//2-1: d//2+1])
        else:
            output_representation = np.zeros((d*d)-1)
            value = patch[d//2, d//2]
        comparison = (patch >= value) * 1
        result = comparison.reshape(-1)
        if d % 2 == 0:
            output_representation = result
        else:
            output_representation[:len(output_representation)//2] = result[:len(output_representation)//2]
            output_representation[len(output_representation)//2:] = result[len(output_representation)//2 + 1:]
        outputs.append(output_representation)
    return np.concatenate(outputs)

def process_data(images):
    results = []
    for image in tqdm(images):
        results.append(binarize_image(image))
    return np.array(results)

train_binary_data = process_data(train_images)
test_binary_data = process_data(test_images)

100%|████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:01<00:00, 1086.40it/s]


In [22]:
from sklearn.neighbors import KNeighborsClassifier

neigh = KNeighborsClassifier(n_neighbors=7)
neigh.fit(train_binary_data, train_labels)
neigh.score(test_binary_data, test_labels)

0.8625

In [12]:
## Ex5
def intersection_kernel(hist1, hist2):
    hist1 = np.expand_dims(hist1, axis=0)
    hist2 = np.expand_dims(hist2, axis=0)
    concatenation = np.concatenate((hist1, hist2),axis=0)
    minimum = np.min(concatenation,axis=0)
    return np.sum(minimum)

def compute_kernel_matrix(histograms1, histograms2, train=True):
    kernel_matrix = np.zeros((len(histograms1), len(histograms2)))
    if train:
        for i in tqdm(range(len(histograms1))):
            for j in range(i, len(histograms2)):
                kernel_matrix[i][j] = intersection_kernel(histograms1[i], histograms2[j])
                kernel_matrix[j][i] = kernel_matrix[i][j]
    else:
        for i in tqdm(range(len(histograms1))):
            for j in range(len(histograms2)):
                kernel_matrix[i][j] = intersection_kernel(histograms1[i], histograms2[j])
    return kernel_matrix
                   
train_matrix = compute_kernel_matrix(train_hist_scaled, train_hist_scaled)
test_matrix = compute_kernel_matrix(train_hist_scaled, test_hist_scaled, train=False)

100%|██████████████████████████████████████████████████████████████████████████████| 5000/5000 [04:19<00:00, 19.27it/s]


In [13]:
clf = SVC(kernel='precomputed')

clf.fit(train_matrix, train_labels)
clf.score(test_matrix.T, test_labels)

0.771